# Cody

## Set up Selenium (get the Chrome driver)

In [2]:
from urllib.request import urlopen
import json
res = urlopen('https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json')
data = json.load(res)
# data

In [6]:
# data['channels'].keys()
data['channels']['Stable']['version']

'114.0.5735.133'

In [12]:
for p in data['channels']['Stable']['downloads']['chrome']:
    print(p['platform'], p['url'])

linux64 https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.133/linux64/chrome-linux64.zip
mac-arm64 https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.133/mac-arm64/chrome-mac-arm64.zip
mac-x64 https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.133/mac-x64/chrome-mac-x64.zip
win32 https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.133/win32/chrome-win32.zip
win64 https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.133/win64/chrome-win64.zip


In [13]:
url_win64 = data['channels']['Stable']['downloads']['chrome'][4]['url']
url_win64

'https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/114.0.5735.133/win64/chrome-win64.zip'

In [14]:
# Get the file name from the url path
import os
os.path.basename(url_win64)

'chrome-win64.zip'

In [15]:
# Download the file
import urllib.request
urllib.request.urlretrieve(url_win64, os.path.basename(url_win64))

('chrome-win64.zip', <http.client.HTTPMessage at 0x259f85ae260>)

In [17]:
# unzip the file
import zipfile
with zipfile.ZipFile(os.path.basename(url_win64), 'r') as zip_ref:
    zip_ref.extractall('chrome-win64')

In [19]:
# Copy the chrome.exe to the current directory
import shutil
shutil.copyfile('chrome-win64/chrome-win64/chrome.exe', 'chrome.exe')

'chrome.exe'

## Get Cody Problems with Selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
driver = webdriver.Chrome()

In [6]:
# Autenticate to Mathworks Account ydebray@mathworks.com
url = "https://www.mathworks.com/login"
driver.get(url)
# driver.find_element(by=By.ID, value="userId")
# driver.find_element(by=By.XPATH, value='//*[@id="userId"]').send_keys("username")

In [7]:
url = "https://www.mathworks.com/matlabcentral/cody/"
driver.get(url)

In [8]:
id = 1
url = f"https://www.mathworks.com/matlabcentral/cody/problems/{id}/solutions/new"
driver.get(url)

In [9]:
css_selector = "[aria-labelledby='learnerProblemDescriptionAriaInfo']"
prompt = driver.find_element(by=By.CSS_SELECTOR, value=css_selector).text
prompt

'Try out this test problem first.\nGiven the variable x as your input, multiply it by two and put the result in y.\nExamples:\n Input  x = 2\n Output y is 4\n Input  x = 17\n Output y is 34'

In [10]:
css_selector2 = "[aria-labelledby='learnerSolutionEditorAriaInfo']"
template = driver.find_element(by=By.CSS_SELECTOR, value=css_selector2).text
template

'function y = times2(x) % Do not edit this line.\n  % Modify the line below so that the output y is twice the incoming value x\n  y = x;\n  % After you modify the code, press the "Submit" button, and you\'re on your way.\nend % Do not edit this line.\n1\n2\n3\n4\n5\n6\n7\n8\n9'

In [12]:
import re

def extract_function_name(matlab_code):
    patterns = [
        r"function\s+\w+\s*=\s*(\w+)\(",
        r"function\s+\w+\s*=\s*(\w+)"
    ]

    for pattern in patterns:
        match = re.search(pattern, matlab_code)
        if match:
            return match.group(1)  # Return the captured function name
        else:
            pattern = r"function\s+\[(\w+(\s+\w+)*)\]\s*=\s*(\w+)\("
            match = re.search(pattern, matlab_code)
            if match:
                return match.group(3)
    return None  # Return None if no match found

# Example usage
matlab_code1 = 'function y = times2(x) % Do not edit this line.\n  % Modify the line below so that the output y is twice the incoming value x\n  y = x;\n  % After you modify the code, press the "Submit" button, and you\'re on your way.\nend % Do not edit this line.\n1\n2\n3\n4\n5\n6\n7\n8\n9'
matlab_code2 = 'function [index1 index2] = nearestNumbers(A)\nindex1 = 1;\nindex2 = 2;'
matlab_code3 = 'function x = oneToTen\n  x = 0;\nend'

function_name1 = extract_function_name(matlab_code1)
function_name2 = extract_function_name(matlab_code2)
function_name3 = extract_function_name(matlab_code3)

print(function_name1)
print(function_name2)
print(function_name3)


times2
nearestNumbers
oneToTen


In [11]:
import re

def extract_function_name(matlab_code):
    # Search for the function name pattern using regular expression
    pattern = r"function\s+\w+\s*=\s*(\w+)\("
    match = re.search(pattern, matlab_code)

    if match:
        return match.group(1)  # Return the captured function name
    else:
        return None  # Return None if no match found

# Example usage
# matlab_code = 'function y = times2(x) % Do not edit this line.\n  % Modify the line below so that the output y is twice the incoming value x\n  y = x;\n  % After you modify the code, press the "Submit" button, and you\'re on your way.\nend % Do not edit this line.\n1\n2\n3\n4\n5\n6\n7\n8\n9'
matlab_code = template

function_name = extract_function_name(matlab_code)
print(function_name)

times2


In [12]:
css_selector3 = "[aria-labelledby='learnerRunYourSolutionEditorAriaInfo']"
scratchpad = driver.find_element(by=By.CSS_SELECTOR, value=css_selector3).text
scratchpad

"% Use this area as a scratchpad to test code that you don't want to submit:\nx = 1 + 2\n% Or call your function:\ntimes2(x)\n% Note this code will NOT be used for submit\n1\n2\n3\n4\n5"

In [13]:
for e in driver.find_elements(by=By.CLASS_NAME,value="problem_test_parent"):
    e.click()

In [16]:
css_selector4 = "[aria-labelledby='learnerSubmissionResultsAriaInfo']"
testcases = driver.find_element(by=By.CSS_SELECTOR, value=css_selector4).text
testcases.split('Test Code:\n')[1:]

['%%\nassert(isequal(times2(1),2));\n1\n2\n3\n4\nTest 2\n',
 '%%\nassert(isequal(times2(11),22));\n1\n2\n3\n4\nTest 3\n',
 '%%\nassert(isequal(times2(-3),-6));\n1\n2\n3\n4\nTest 4\n',
 '%%\nassert(isequal(times2(29),58));\n1\n2']

In [43]:
testsuite = [t.split('\n1')[0].replace('%%\n',f'%% test {i+1}\n') for i,t in enumerate(testcases.split('Test Code:\n')[1:])]
testsuite

['%% test 1\nassert(isequal(times2(1),2));',
 '%% test 2\nassert(isequal(times2(11),22));',
 '%% test 3\nassert(isequal(times2(-3),-6));',
 '%% test 4\nassert(isequal(times2(29),58));']

In [136]:
# create a directory for the problem
import os
os.mkdir(f'problem{id}')

In [44]:
# save the test suite in the directory
with open(f'problem{id}/{function_name}Test.m', 'w') as f:
    f.write('\n'.join(testsuite))

In [119]:
# other approaches
driver.find_element(by=By.CLASS_NAME, value="learner_code_fields").text
# driver.find_element(by=By.CLASS_NAME, value="lti_editor").text
# driver.find_element(by=By.CLASS_NAME, value="mwRichTextComponentDocument").text

'function y = times2(x) % Do not edit this line.\n  % Modify the line below so that the output y is twice the incoming value x\n  y = x;\n  % After you modify the code, press the "Submit" button, and you\'re on your way.\nend % Do not edit this line.\n1\n2\n3\n4\n5\n6\n7\n8\n9'

In [110]:
# driver.quit()

## Solve with GPT-3.5

In [99]:
# Manual entry of the prompt and template
prompt = """
Try out this test problem first.
Given the variable x as your input, multiply it by two and put the result in y.
Examples:
 Input  x = 2
 Output y is 4

 Input  x = 17
 Output y is 34
"""
template = """
function y = times2(x) % Do not edit this line.

  % Modify the line below so that the output y is twice the incoming value x

  y = x;

  % After you modify the code, press the "Submit" button, and you're on your way.

end % Do not edit this line.
"""
scratchpad = """
% Use this area as a scratchpad to test code that you don't want to submit:
x = 1 + 2
% Or call your function:
times2(x)
% Note this code will NOT be used for submit
"""
function_name = 'times2'

In [19]:
import openai
import tomli
with open('secrets.toml','rb') as f:
    toml_dict = tomli.load(f)
openai.api_key = toml_dict['OPEN_AI_KEY']

In [20]:
messages = [
            {"role": "system", "content": "You are a helpful assistant that solves coding problems in MATLAB. reply only with the MATLAB code, without formatting or chit chat. Format the result as a function"}, 
            {"role": "user", "content": "This is the problem statement:\n"+prompt+"\n Integrate the result in the following function:\n"+template},
            ]

In [21]:
res = openai.ChatCompletion.create(messages=messages, model="gpt-3.5-turbo")
solution = res['choices'][0]['message']['content']
print(solution)

function y = times2(x) % Do not edit this line.
  % Modify the line below so that the output y is twice the incoming value x
  y = 2 * x;
  % After you modify the code, press the "Submit" button, and you're on your way.
end % Do not edit this line.


In [22]:
with open(f'problem{id}/{function_name}.m','w') as f:
    f.write(solution)

## Test code with MATLAB

https://www.mathworks.com/help/matlab/matlab_prog/write-script-based-unit-tests.html

In [23]:
import matlab.engine
eng = matlab.engine.start_matlab()

In [24]:
eng.cd(f'problem{id}')
# eng.cd('..')

'C:\\Users\\ydebray\\Downloads\\cody'

In [25]:
eng.feval(function_name, 2) # function_name = 'times2'

4

In [37]:
# execute the test suite
results = eng.runtests()

In [39]:
eng.workspace['results'] = results

In [40]:
logOut = eng.evalc('results')
from IPython.core.display import HTML
HTML(logOut.replace('\n','<br>'))

In [45]:
logOut = eng.evalc('rt = table(results)')
HTML(logOut.replace('\n','<br>'))

In [46]:
rt = eng.table(results)

In [48]:
eng.writetable(rt,function_name+".csv",nargout=0)

In [ ]:
# for id in range(1,50):
#     try:
#         eng.cd(f'cody/problem{id}/gpt-3.5-turbo')
#         results = eng.runtests()
#         rt = eng.table(results)
#         eng.workspace['rt'] = rt
#         eng.evalc("fileID = fopen('result.txt', 'w');if any(rt.Passed == false); fprintf(fileID, 'failed'); else; fprintf(fileID, 'passed'); end;fclose(fileID);")
#         status = eng.evalc("if any(rt.Passed == false); disp('failed'); else; disp('passed'); end;")
#         print(f'Problem {id} tested: {status}')
#         eng.cd('../../..')
#     except:
#         print(f'Problem {id} failed')
#         pass

In [41]:
eng.desktop(nargout=0)

In [156]:
eng.exit()